In [1]:
## fix auto reloading
import pandas
import os
import sys
from dotenv import load_dotenv
import json
import pandas as pd
from video_selection.google_trends import get_trends
from video_selection.search_by_keyword import write_keyword_and_id
from video_selection.filter import filter_sample_videos
from feature_extraction.metadata import metadata_extraction
from feature_extraction.metadata import youtube_authenticate
from feature_extraction.video_analysis import analyze_by_path
from feature_extraction.video_preprocess import *
from feature_extraction.transcription import transcribe_gcs
from feature_extraction.NLP import *
from datetime import datetime

# Load environment variables from the .env file
load_dotenv(override=True)

True

# Video Selection

In [7]:
# pipeline setup
keyword_path = './input/input_keyword_list.csv' # Modified for different disease, Input the keyword list
keyword_ref = 'sleep apnea causes' # use the first keyword for comparison in each iteration. 
num_of_result = 100

In [ ]:
# test for google trends
keyword_list = get_trends(keyword_path, keyword_ref, num_of_result)
print(keyword_list[:10])

In [ ]:
# test search for keyword
df_keyword = write_keyword_and_id(keyword_list, 5)
df_keyword.head(5)

In [ ]:
df_keyword

,keyword,id,rank,toKeep
304,excessive yawning,-MjaUhSOf94,5,False
367,difficulty falling asleep,PmM1IH-LaVQ,3,False
394,Obstructive Syndrome,krxcVBgerHM,5,False
480,pain in front center of brain,FSPzEzNBtMg,1,False
481,pain in front center of brain,HvVa7vUSRIo,2,False
482,pain in front center of brain,1JrdO-1Wf8g,3,False
483,pain in front center of brain,TYhgbE_84MA,4,False
484,pain in front center of brain,vPje60GgHXc,5,False


In [ ]:
# test filter and sample
credential_path = "oauth_credential.json"
# just for test
df_keyword = pd.read_csv('./temp/videoID_list.csv')
sampled_videos = filter_sample_videos(df_keyword, credential_path)
sampled_videos.head(10)

In [ ]:
# display the video_dictionary
df_video = pd.read_csv('temp/sampled_filtered_video_list.csv')
df_video.head()

,keyword,id,rank
0,ACROMEGALY,KLoaYPXlWHM,22
1,ACROMEGALY,OFl_bKVYsPo,9
2,ACROMEGALY,DAWYyBQJ8Jk,7
3,ACROMEGALY,AI-edvZx2GU,23
4,Anxious mood,YufQY6HqCW0,4


In [ ]:
df_video[df_video['id']== "KLoaYPXlWHM"].index[0]

0

# Extract Metadata

In [2]:
# pipeline setup
testID = "KLoaYPXlWHM"
test_keyword = "ACROMEGALY"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getcwd() + os.environ.get("SERVICE_ACCOUNT_PATH")
youtube = youtube_authenticate(os.environ.get("OAUTH_CREDENTIAL_PATH"))
local_video_path = "/Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/video"
local_audio_path = "/Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio"
bucket_name = "youtube_pipeline_testing"
gcs_bucket_path = os.path.join("gs://", bucket_name)
gcs_video_path = os.path.join(gcs_bucket_path, testID +".mp4")
gcs_audio_path = os.path.join(gcs_bucket_path, testID +".wav")

In [3]:
# video preprocess
download_video_by_id(testID, local_video_path)
extract_audios_from_videos(local_video_path, local_audio_path)
upload_files(bucket_name, local_video_path)
upload_files(bucket_name, local_audio_path)
# bucket_metadata(bucket_name)

Downloading: https://www.youtube.com/watch?v=KLoaYPXlWHM
Video successfully downloaded!
MoviePy - Writing audio in /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio/KLoaYPXlWHM.wav


MoviePy - Done.
File /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/video/KLoaYPXlWHM.mp4 uploaded to KLoaYPXlWHM.mp4.
File /Users/ZHANGRY/Codes/youtube-analytics-pipeline/temp/audio/KLoaYPXlWHM.wav uploaded to KLoaYPXlWHM.wav.


In [4]:
# raw metadata features
raw_metadata = metadata_extraction(youtube, testID) # in json file
pd.DataFrame(raw_metadata, index=[0])

extracting the id:  KLoaYPXlWHM
Retrieving accTag for:  https://www.youtube.com/watch?v=KLoaYPXlWHM
error in extracting accTag


,id,hasTags,num_of_tags,hasDescription,hasThumbnail,channel_subscribers,accreditationTag,duration,description,publish_days
0,KLoaYPXlWHM,1,17,1,1,33400,0,259.0,Part II of The Voices of Acromegaly video seri...,2998


In [10]:
# video feature, 4 minutes for a video
video_feature = analyze_by_path(gcs_video_path)
pd.DataFrame(video_feature, index=[0])


Processing video for shot change annotations:

Finished processing.

Processing video for object annotations.

Finished processing.


Processing video for text detection.


,id,num_of_shots,num_of_objects,text_confidence
0,KLoaYPXlWHM.mp4,9,145,0.97195


In [5]:
# transcription feature, 1 min 30 s for a video
audio_feature = transcribe_gcs(gcs_audio_path)
pd.DataFrame(audio_feature, index=[0])

Waiting for operation to complete...
Transcript: the voices of acromegaly a 360-degree look at that condition part to managing acromegaly A Day in the Life it's challenging to manage acromegaly patients should work with their Physicians to regularly Monitor and control their g h and igf-1 levels to help reduce the potential Health complications of acromegaly such as heart disease hypertension or diabetes there are also often psychosocial effects associated with this rare and life-threatening condition listen to the following people describe the day-to-day struggles of living with acromegaly the challenges faced and the adjustments they've had to make in order to cope with this condition
Confidence: 0.9662675261497498
Transcript:  say this a lot of stress and arranged with the diagnosis of start with another length of time it takes to get the diagnosis and then you faced with possibly surgery maybe rages therapy you know that's very frightening and you know what does that mean going int

,id,transcription,transcription confidence
0,KLoaYPXlWHM.wav,the voices of acromegaly a 360-degree look at ...,0.935412


In [12]:
# nlp feature
description = raw_metadata['description']
transcription = audio_feature['transcription']
print(count_paragraph_stats(description))

(44, 34, 7, 30, 0, 0, 8.581266233766232)


In [14]:
print(medical_entity_count(description))

0
